### GPU Settings

In [ ]:
# 1. GPU ရွေးမယ်
# Edit => Notebook Settings မှာ GPU ကို ရွေးပေးရမယ် မဟုတ်ရင် Run ရတာ အသေကြာနေလိမ့်မယ်
# Checking whether the GPU Device is selected or not
%tensorflow_version 2.x
import tensorflow as tf             #tf library for deep learning
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
%time


### Import Necessary Libiaries

In [ ]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop        #op is to better results
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import drive

### Connecting with Google Drive

In [ ]:
# 3. google drive နဲ့ ချိတ်ပါ

from google.colab import drive
drive.mount('/content/drive')

### Data Understanding and Preparation

In [ ]:
# 4. Directory of Data Folders
# မိမိရဲ့ ပတ်လမ်းကြောင်း ပြောင်းဖို့ မမေ့ပါနဲ့

train_dir  = '/content/drive/MyDrive/AI-Simbolo/Deep Learning/Dogs vs Cats vs Tigers/Training datasets' 
test_dir   = '/content/drive/MyDrive/AI-Simbolo/Deep Learning/Dogs vs Cats vs Tigers/Testing datasets'

In [ ]:
# ပုံတွေရဲ့ Range  ကို scale လုပ်တာ။
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )        #(0-255) into scale range is 0 and 1, cuz minimize scale.
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
import os 
os.listdir(train_dir)   #training folder ေအာက်မှာဘာတွေရှိလဲကြည့် 

In [ ]:
os.listdir(train_dir+'/cats')   #cats folder ထဲက image တွေထုတ်ကြည့်

In [ ]:
import cv2        #for image reading with cv2 library
image = cv2.imread(train_dir+'/cats/'+'image.jpeg')  #
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #BGR to RGB ေပြာင်း 

In [ ]:
import matplotlib.pyplot as plt   #matplotlib is good for map and chart
plt.imshow(image)   

In [ ]:
# ပုံတစ်ပုံကို ထုတ်ကြည့်ထားတာ
plt.imshow(image_rgb)   #original image with RGB

In [ ]:
import numpy as np
np.array(image_rgb)

In [ ]:
batch_size = 5
 
# ကိုယ်ယူတဲ့ Target Size တွေကို မှတ်ထားပါ နောက်ပြန်သုံးမှာမို့
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size=(100, 100),           #all images ကို width height ညှိပေးတာ
                                                 batch_size=batch_size,
                                                 class_mode='categorical')    #there are 3 categories (cats,dogs,tigers) 
                                                 #class_mode='binary')  #if there are two categories, use binary.
 
test_set = test_datagen .flow_from_directory(test_dir,
                                            target_size=(100, 100),
                                            batch_size=batch_size,
                                            class_mode='categorical')
                                            #class_mode='binary')

### Modeling

In [ ]:
# 5. Neural Network Implemenation
""" နောက်ဆုံး Layer မှာ ကိုယ့်ရဲ့ Label အရေအတွက် အတိုင်းထည့်ပေးရမှာပါ။ 
    Label က ငါးခုဆိုရင် tf.keras.layers.Dense(5,activation=tf.nn.softmax)
 
    Label က နှစ်ခုဆိုရင်တော့ tf.keras.layers.Dense(1,activation=tf.nn.sigmoid)
    Activation ကို sigmoid ပြောင်းသုံးသင့်ပါတယ်။

    Label က နှစ်ခုနဲ့အထက်ဆိုရင်တော့ (n = more than two classes) ဆိုရင်တော့ tf.keras.layers.Dense(n,activation=tf.nn.softmax)
    Activation ကို softmax ပြောင်းသုံးသင့်ပါတယ်။
"""
# 
import tensorflow as tf
 
model  = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape = (100, 100, 3)),    #flatten က matrix table ကို row တခုထဲေပြာင်း, 3 က 3 channels RGB
        # tf.keras.layers.Dense(1000,activation=tf.nn.relu),#hidden
        # tf.keras.layers.Dense(500,activation=tf.nn.relu),
        # tf.keras.layers.Dense(200,activation=tf.nn.relu),
        tf.keras.layers.Dense(100,activation=tf.nn.relu),  #hidden layer 100 neurons ထား 
        tf.keras.layers.Dense(3,activation=tf.nn.softmax) # output layers with 3 labels (neurons)
])

In [ ]:
# label က နှစ်ခုပဲဆိုရင်တော့ binary_crossentropy ကိုသုံးသင့်ပါတယ်။
# label က နှစ်ခုအထက် ဆိုရင်တော့ categorical_crossentropyကိုသုံးသင့်ပါတယ်။
model.compile(optimizer='adam',   #result ပိုေကာင်းအောင်ထည့်တာ 
              loss = 'categorical_crossentropy',
              metrics =['acc'])   #accuracy 

In [ ]:
# Data Training
#7414 မှာ ကိုယ့်ရဲ့ ပုံအရေအတွက် ပြောင်းဖို့မမေ့ပါနဲ့
# Edit 7414 depending on your data
history = model.fit(training_set,
                              #validation_data = validation_set,
                              steps_per_epoch=15//batch_size,  #epoch (run တဲ့အကြိမ်အရေအတွက်) တခုမှာ ၁၅ကြိမ်သွားမယ် 
                        
                         epochs=6,                   #count of runnning
                         #validation_steps=validation_length//batch_size,
                         #validation_steps=1000//batch_size,
                         verbose = 2,
                         shuffle = False)  #data တွေကို random/shuffle လုပ်ပြီး run 

In [ ]:
# Accuracyနဲ့ loss တန်ဖိုးတွေက epoch အလိုက်ထုတ်ကြည့်ထားတာ
import matplotlib.pyplot as plt
acc = history.history[     'acc' ]
#val_acc  = history.history[ 'val_acc' ]
loss     = history.history[    'loss' ]
#val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) 
plt.plot  ( epochs,     acc ,label = 'Accuracy')
#plt.plot  ( epochs, val_acc)
#plt.title ('Training and validation accuracy')
plt.title ('Training Accuracy')
plt.figure()

plt.plot  ( epochs,     loss )
#plt.plot  ( epochs, val_loss )
#plt.title ('Training and validation loss')
plt.title ('Training Loss')

plt.show()

### Evaluation

In [ ]:
# Class Labels
class_dictionary = test_set.class_indices
print('Labels dictionary',class_dictionary)  #evaluation စမ်းရင် unseen testing data နဲ့စမ်း 

In [ ]:
y_predicted = model.predict(test_set) # A way to predict data

In [ ]:
y_predicted

In [ ]:
#  တစ်ပုံချင်းစီကို Testလုပ်တာ
folder_names = ['cats','dogs', 'tigers'] # Please don't forget to change here
y_pred = []
for folder in folder_names:
    path = test_dir+"/"+folder
    path_fnames = os.listdir(path)
    for i in path_fnames:
        path2 = path+'/'+i
        img = tf.keras.preprocessing.image.load_img(path2, target_size=(28,28))#target size ကို သတိထားပါ
        x = tf.keras.preprocessing.image.img_to_array(img)
        x = np.expand_dims(x,axis=0)
        x /= 255.0
        images = np.vstack([x])# [1 2 3 4 5 6]
        classes = model.predict(x)
        y_classes=classes.argmax(axis=-1)
        y_pred.append(y_classes[0])
    print()

In [ ]:
# Actual Labels
y_true = test_set.classes.tolist()
print(y_true)

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_true,y_pred)
print('confusion_matrix')
print(confusion_matrix)

In [ ]:
y_true

In [ ]:
print(len(y_true))

In [ ]:
y_pred

In [ ]:
print(len(y_pred))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

print('Accuracy Score',accuracy_score(y_true, y_pred)*100,'%')
print('Precision Macro Score ',precision_score(y_true, y_pred,average = 'macro')*100,'%')
print('Recall_Score',recall_score(y_true, y_pred, average = 'macro')*100,'%')
print('F1_Score',f1_score(y_true, y_pred, average = 'macro')*100,'%')

### Saving Model File

In [ ]:
# model ကို သိမ်းတာကလည်း အရေးကြီးတယ်။
model.save('/neural_networks.h5')

### Implement with Streamlit

In [ ]:
# How to implement your model with streamlit: https://github.com/ThuraAung1601